In [2]:
%load_ext google.cloud.bigquery
import pandas as pd

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


This will select data from BQ into a dataframe

In [12]:
%%bigquery prices
SELECT * FROM `kaggle-covid.sell_prices.sell_prices`

In [14]:
prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_396,11348,10.98
6841117,WI_3,FOODS_3_396,11349,10.98
6841118,WI_3,FOODS_3_396,11350,10.98
6841119,WI_3,FOODS_3_396,11351,10.98


In [3]:
%%bigquery calendar
SELECT * FROM `kaggle-covid.calendar.calendar`


In [4]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,None,None,None,None,0,0,0
1,2011-02-19,11104,Saturday,1,2,2011,d_22,None,None,None,None,0,0,0
2,2011-02-26,11105,Saturday,1,2,2011,d_29,None,None,None,None,0,0,0
3,2011-03-19,11108,Saturday,1,3,2011,d_50,None,None,None,None,0,0,0
4,2011-03-26,11109,Saturday,1,3,2011,d_57,None,None,None,None,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-03-11,11606,Friday,7,3,2016,d_1869,None,None,None,None,0,1,1
1965,2016-04-08,11610,Friday,7,4,2016,d_1897,None,None,None,None,1,0,1
1966,2016-04-15,11611,Friday,7,4,2016,d_1904,None,None,None,None,0,1,1
1967,2016-05-06,11614,Friday,7,5,2016,d_1925,None,None,None,None,1,1,1


In [5]:
%%bigquery item_013
-- select one item in one store
select 
*
from
sales_train_evaluation.sales_train_evaluation a
where a.store_id = 'CA_1' and a.item_id = 'HOBBIES_1_013';

In [6]:
item_013

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_013_CA_1_evaluation,HOBBIES_1_013,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,1,0,0,0,0,0,1,1


In [7]:
%%bigquery store_CA_1
-- select all items from one store
select 
id, a.item_id, dept_id,cat_id, a.store_id, state_id, d_1, d_1941
from
sales_train_evaluation.sales_train_evaluation a
where a.store_id = 'CA_1';

In [8]:
store_CA_1

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,1
3,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0
4,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0
...,...,...,...,...,...,...,...,...
3044,FOODS_3_319_CA_1_evaluation,FOODS_3_319,FOODS_3,FOODS,CA_1,CA,5,0
3045,FOODS_3_681_CA_1_evaluation,FOODS_3_681,FOODS_3,FOODS,CA_1,CA,12,45
3046,FOODS_3_080_CA_1_evaluation,FOODS_3_080,FOODS_3,FOODS,CA_1,CA,33,21
3047,FOODS_3_586_CA_1_evaluation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,42,43


Pivot all the date columns into rows to get something more amenable for time series prediction 

In [9]:
df = pd.melt(
    store_CA_1,
    id_vars=[
        'id',
        'item_id',
        'dept_id',
        'cat_id',
        'store_id',
        'state_id'
    ],
    var_name='d',
    value_name='items_sold'
).dropna()

In [10]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
6093,FOODS_3_319_CA_1_evaluation,FOODS_3_319,FOODS_3,FOODS,CA_1,CA,d_1941,0
6094,FOODS_3_681_CA_1_evaluation,FOODS_3_681,FOODS_3,FOODS,CA_1,CA,d_1941,45
6095,FOODS_3_080_CA_1_evaluation,FOODS_3_080,FOODS_3,FOODS,CA_1,CA,d_1941,21
6096,FOODS_3_586_CA_1_evaluation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_1941,43


Join with calendar dataframe

In [11]:
df = pd.merge(df, calendar, on='d', how='left')
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,None,None,None,None,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,None,None,None,None,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,None,None,None,None,0,0,0
3,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,None,None,None,None,0,0,0
4,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,None,None,None,None,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6093,FOODS_3_319_CA_1_evaluation,FOODS_3_319,FOODS_3,FOODS,CA_1,CA,d_1941,0,2016-05-22,11617,...,2,5,2016,None,None,None,None,0,0,0
6094,FOODS_3_681_CA_1_evaluation,FOODS_3_681,FOODS_3,FOODS,CA_1,CA,d_1941,45,2016-05-22,11617,...,2,5,2016,None,None,None,None,0,0,0
6095,FOODS_3_080_CA_1_evaluation,FOODS_3_080,FOODS_3,FOODS,CA_1,CA,d_1941,21,2016-05-22,11617,...,2,5,2016,None,None,None,None,0,0,0
6096,FOODS_3_586_CA_1_evaluation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_1941,43,2016-05-22,11617,...,2,5,2016,None,None,None,None,0,0,0


Now join with item prices for each item within store for that date (wm_yrwk).

In [15]:
df = pd.merge(
    df,
    prices,
    on=['store_id','item_id','wm_yr_wk'],
    how='left') 

In [30]:
df.describe

<bound method NDFrame.describe of                                  id        item_id    dept_id   cat_id  \
0     HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1     HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2     HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3     HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
4     HOBBIES_1_007_CA_1_evaluation  HOBBIES_1_007  HOBBIES_1  HOBBIES   
...                             ...            ...        ...      ...   
6093    FOODS_3_319_CA_1_evaluation    FOODS_3_319    FOODS_3    FOODS   
6094    FOODS_3_681_CA_1_evaluation    FOODS_3_681    FOODS_3    FOODS   
6095    FOODS_3_080_CA_1_evaluation    FOODS_3_080    FOODS_3    FOODS   
6096    FOODS_3_586_CA_1_evaluation    FOODS_3_586    FOODS_3    FOODS   
6097    FOODS_3_587_CA_1_evaluation    FOODS_3_587    FOODS_3    FOODS   

     store_id state_id       d  items_sold        date  wm_yr_wk  ... month  

In [16]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
3,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
4,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN


In [24]:
df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            6088, 6089, 6090, 6091, 6092, 6093, 6094, 6095, 6096, 6097],
           dtype='int64', length=6098)

In [29]:
df.loc[df['wm_yr_wk'] == 11101]

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
3,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
4,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,FOODS_3_319_CA_1_evaluation,FOODS_3_319,FOODS_3,FOODS,CA_1,CA,d_1,5,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,1.00
3045,FOODS_3_681_CA_1_evaluation,FOODS_3_681,FOODS_3,FOODS,CA_1,CA,d_1,12,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,1.25
3046,FOODS_3_080_CA_1_evaluation,FOODS_3_080,FOODS_3,FOODS,CA_1,CA,d_1,33,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,1.48
3047,FOODS_3_586_CA_1_evaluation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_1,42,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,1.48


In [ ]:
HOBBIES_1_008